<a href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/tour-of-beam/reading-and-writing-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# Reading and writing data -- _Tour of Beam_

So far we've learned some of the basic transforms like
[`Map`](https://beam.apache.org/documentation/transforms/python/elementwise/map),
[`FlatMap`](https://beam.apache.org/documentation/transforms/python/elementwise/flatmap),
[`Filter`](https://beam.apache.org/documentation/transforms/python/elementwise/filter),
[`Combine`](https://beam.apache.org/documentation/transforms/python/aggregation/combineglobally), and
[`GroupByKey`](https://beam.apache.org/documentation/transforms/python/aggregation/groupbykey).
These allow us to transform data in any way, but so far we've used
[`Create`](https://beam.apache.org/documentation/transforms/python/other/create)
to get data from an in-memory
[`iterable`](https://docs.python.org/3/glossary.html#term-iterable), like a `list`.

This works well for experimenting with small datasets. For larger datasets we can use `Source` transforms to read data and `Sink` transforms to write data.
If there are no built-in `Source` or `Sink` transforms, we can also easily create our custom I/O transforms.

Let's create some data files and see how we can read them in Beam.

In [ ]:
# Install apache-beam with pip.
!pip install --quiet apache-beam

# Create a directory for our data files.
!mkdir -p data

In [ ]:
%%writefile data/my-text-file-1.txt
This is just a plain text file, UTF-8 strings are allowed 🎉.
Each line in the file is one element in the PCollection.

In [ ]:
%%writefile data/my-text-file-2.txt
There are no guarantees on the order of the elements.
ฅ^•ﻌ•^ฅ

In [ ]:
%%writefile data/penguins.csv
species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0.2545454545454545,0.6666666666666666,0.15254237288135594,0.2916666666666667
0,0.26909090909090905,0.5119047619047618,0.23728813559322035,0.3055555555555556
1,0.5236363636363636,0.5714285714285713,0.3389830508474576,0.2222222222222222
1,0.6509090909090909,0.7619047619047619,0.4067796610169492,0.3333333333333333
2,0.509090909090909,0.011904761904761862,0.6610169491525424,0.5
2,0.6509090909090909,0.38095238095238104,0.9830508474576272,0.8333333333333334

# Reading from text files

We can use the
[`ReadFromText`](https://beam.apache.org/releases/pydoc/current/apache_beam.io.textio.html#apache_beam.io.textio.ReadFromText)
transform to read text files into `str` elements.

It takes a
[_glob pattern_](https://en.wikipedia.org/wiki/Glob_%28programming%29)
as an input, and reads all the files that match that pattern.
It returns one element for each line in the file.

For example, in the pattern `data/*.txt`, the `*` is a wildcard that matches anything. This pattern matches all the files in the `data/` directory with a `.txt` extension.

In [96]:
import apache_beam as beam

input_files = 'data/*.txt'
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read files' >> beam.io.ReadFromText(input_files)
      | 'Print contents' >> beam.Map(print)
  )

There are no guarantees on the order of the elements.
ฅ^•ﻌ•^ฅ
This is just a plain text file, UTF-8 strings are allowed 🎉.
Each line in the file is one element in the PCollection.


# Writing to text files

We can use the
[`WriteToText`](https://beam.apache.org/releases/pydoc/2.27.0/apache_beam.io.textio.html#apache_beam.io.textio.WriteToText) transform to write `str` elements into text files.

It takes a _file path prefix_ as an input, and it writes the all `str` elements into one or more files with filenames starting with that prefix. You can optionally pass a `file_name_suffix` as well, usually used for the file extension. Each element goes into its own line in the output files.

In [ ]:
import apache_beam as beam

output_file_name_prefix = 'outputs/file'
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Create file lines' >> beam.Create([
          'Each element must be a string.',
          'It writes one element per line.',
          'There are no guarantees on the line order.',
          'The data might be written into multiple files.',
      ])
      | 'Write to files' >> beam.io.WriteToText(
          output_file_name_prefix,
          file_name_suffix='.txt')
  )

In [98]:
# Lets look at the output files and contents.
!head outputs/file*.txt

Each element must be a string.
It writes one element per line.
There are no guarantees on the line order.
The data might be written into multiple files.


# Reading data

Your data might reside in various input formats. Take a look at the
[Built-in I/O Transforms](https://beam.apache.org/documentation/io/built-in)
page for a list of all the available I/O transforms in Beam.

If none of those work for you, you might need to create your own input transform.

> ℹ️ For a more in-depth guide, take a look at the
[Developing a new I/O connector](https://beam.apache.org/documentation/io/developing-io-overview) page.

## Reading from an `iterable`

The easiest way to create elements is using
[`FlatMap`](https://beam.apache.org/documentation/transforms/python/elementwise/flatmap).

A common way is having a [`generator`](https://docs.python.org/3/glossary.html#term-generator) function. This could take an input and _expand_ it into a large amount of elements. The nice thing about `generator`s is that they don't have to fit everything into memory like a `list`, they simply
[`yield`](https://docs.python.org/3/reference/simple_stmts.html#yield)
elements as they process them.

For example, let's define a `generator` called `count`, that `yield`s the numbers from `0` to `n`. We use `Create` for the initial `n` value(s) and then exapand them with `FlatMap`.

In [8]:
import apache_beam as beam

def count(n):
  for i in range(n):
    yield i

n = 5
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Create inputs' >> beam.Create([n])
      | 'Generate elements' >> beam.FlatMap(count)
      | 'Print elements' >> beam.Map(print)
  )

0
1
2
3
4


## Creating an input transform

For a nicer interface, we could abstract the `Create` and the `FlatMap` into a custom `PTransform`. This would give a more intuitive way to use it, while hiding the inner workings.

We create a new class that inherits from `beam.PTransform`. Any input from the generator function, like `n`, becomes a class field. The generator function itself would now become a
[`staticmethod`](https://docs.python.org/3/library/functions.html#staticmethod).
And we can hide the `Create` and `FlatMap` in the `expand` method.

Now we can use our transform in a more intuitive way, just like `ReadFromText`.

In [9]:
import apache_beam as beam

class Count(beam.PTransform):
  def __init__(self, n):
    self.n = n

  @staticmethod
  def count(n):
    for i in range(n):
      yield i

  def expand(self, pcollection):
    return (
        pcollection
        | 'Create inputs' >> beam.Create([self.n])
        | 'Generate elements' >> beam.FlatMap(Count.count)
    )

n = 5
with beam.Pipeline() as pipeline:
  (
      pipeline
      | f'Count to {n}' >> Count(n)
      | 'Print elements' >> beam.Map(print)
  )

0
1
2
3
4


## Example: Reading CSV files

Lets say we want to read CSV files to get elements as Python dictionaries. We like how `ReadFromText` expands a file pattern, but we might want to allow for multiple patterns as well.

We create a `ReadCsvFiles` transform, which takes a list of `file_patterns` as input. It expands all the `glob` patterns, and then, for each file name it reads each row as a `dict` using the
[`csv.DictReader`](https://docs.python.org/3/library/csv.html#csv.DictReader) module.

In [86]:
import apache_beam as beam
import csv
import glob

class ReadCsvFiles(beam.PTransform):
  def __init__(self, file_patterns):
    self.file_patterns = file_patterns

  @staticmethod
  def read_csv_lines(file_name):
    with open(file_name, 'r') as f:
      for row in csv.DictReader(f):
        yield dict(row)

  def expand(self, pcollection):
    return (
        pcollection
        | 'Create file patterns' >> beam.Create(self.file_patterns)
        | 'Expand file patterns' >> beam.FlatMap(glob.glob)
        | 'Read CSV lines' >> beam.FlatMap(self.read_csv_lines)
    )

input_patterns = ['data/*.csv']
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read CSV files' >> ReadCsvFiles(input_patterns)
      | 'Print elements' >> beam.Map(print)
  )

{'species': '0', 'culmen_length_mm': '0.2545454545454545', 'culmen_depth_mm': '0.6666666666666666', 'flipper_length_mm': '0.15254237288135594', 'body_mass_g': '0.2916666666666667'}
{'species': '0', 'culmen_length_mm': '0.26909090909090905', 'culmen_depth_mm': '0.5119047619047618', 'flipper_length_mm': '0.23728813559322035', 'body_mass_g': '0.3055555555555556'}
{'species': '1', 'culmen_length_mm': '0.5236363636363636', 'culmen_depth_mm': '0.5714285714285713', 'flipper_length_mm': '0.3389830508474576', 'body_mass_g': '0.2222222222222222'}
{'species': '1', 'culmen_length_mm': '0.6509090909090909', 'culmen_depth_mm': '0.7619047619047619', 'flipper_length_mm': '0.4067796610169492', 'body_mass_g': '0.3333333333333333'}
{'species': '2', 'culmen_length_mm': '0.509090909090909', 'culmen_depth_mm': '0.011904761904761862', 'flipper_length_mm': '0.6610169491525424', 'body_mass_g': '0.5'}
{'species': '2', 'culmen_length_mm': '0.6509090909090909', 'culmen_depth_mm': '0.38095238095238104', 'flipper_l

## Example: Reading from a SQLite database

Lets begin by creating a small SQLite local database file.

Run the _"Creating the SQLite database"_ cell to create a new SQLite3 database with the filename you choose. You can double-click it to see the source code if you want.

In [11]:
#@title Creating the SQLite database
import sqlite3

databse_file = "moon-phases.db" #@param {type:"string"}

with sqlite3.connect(databse_file) as db:
  cursor = db.cursor()

  # Create the moon_phases table.
  cursor.execute('''
    CREATE TABLE IF NOT EXISTS moon_phases (
      id INTEGER PRIMARY KEY,
      phase_emoji TEXT NOT NULL,
      peak_datetime DATETIME NOT NULL,
      phase TEXT NOT NULL)''')

  # Truncate the table if it's already populated.
  cursor.execute('DELETE FROM moon_phases')

  # Insert some sample data.
  insert_moon_phase = 'INSERT INTO moon_phases(phase_emoji, peak_datetime, phase) VALUES(?, ?, ?)'
  cursor.execute(insert_moon_phase, ('🌕', '2017-12-03 15:47:00', 'Full Moon'))
  cursor.execute(insert_moon_phase, ('🌗', '2017-12-10 07:51:00', 'Last Quarter'))
  cursor.execute(insert_moon_phase, ('🌑', '2017-12-18 06:30:00', 'New Moon'))
  cursor.execute(insert_moon_phase, ('🌓', '2017-12-26 09:20:00', 'First Quarter'))
  cursor.execute(insert_moon_phase, ('🌕', '2018-01-02 02:24:00', 'Full Moon'))
  cursor.execute(insert_moon_phase, ('🌗', '2018-01-08 22:25:00', 'Last Quarter'))
  cursor.execute(insert_moon_phase, ('🌑', '2018-01-17 02:17:00', 'New Moon'))
  cursor.execute(insert_moon_phase, ('🌓', '2018-01-24 22:20:00', 'First Quarter'))
  cursor.execute(insert_moon_phase, ('🌕', '2018-01-31 13:27:00', 'Full Moon'))

  # Query for the data in the table to make sure it's populated.
  cursor.execute('SELECT * FROM moon_phases')
  for row in cursor.fetchall():
    print(row)

(1, '🌕', '2017-12-03 15:47:00', 'Full Moon')
(2, '🌗', '2017-12-10 07:51:00', 'Last Quarter')
(3, '🌑', '2017-12-18 06:30:00', 'New Moon')
(4, '🌓', '2017-12-26 09:20:00', 'First Quarter')
(5, '🌕', '2018-01-02 02:24:00', 'Full Moon')
(6, '🌗', '2018-01-08 22:25:00', 'Last Quarter')
(7, '🌑', '2018-01-17 02:17:00', 'New Moon')
(8, '🌓', '2018-01-24 22:20:00', 'First Quarter')
(9, '🌕', '2018-01-31 13:27:00', 'Full Moon')


We could use a `FlatMap` transform to receive a SQL query and `yield` each result row, but that would mean creating a new database connection for each query. If we generated a large number of queries, creating that many connections could be a bottleneck.

It would be nice to create the database connection only once for each worker, and every query could use the same connection if needed.

We can use a
[custom `DoFn` transform](https://beam.apache.org/documentation/transforms/python/elementwise/pardo/#example-3-pardo-with-dofn-methods)
for this. It allows us to open and close resources, like the database connection, only _once_ per `DoFn` _instance_ by using the `setup` and `teardown` methods.

> ℹ️ It should be safe to _read_ from a database with multiple concurrent processes using the same connection, but only one process should be _writing_ at once.

In [12]:
import apache_beam as beam
import sqlite3

class SQLiteSelect(beam.DoFn):
  def __init__(self, database_file):
    self.database_file = database_file
    self.connection = None

  def setup(self):
    self.connection = sqlite3.connect(self.database_file)

  def process(self, query):
    table, columns = query
    cursor = self.connection.cursor()
    cursor.execute(f"SELECT {','.join(columns)} FROM {table}")
    for row in cursor.fetchall():
      yield dict(zip(columns, row))

  def teardown(self):
    self.connection.close()

class SelectFromSQLite(beam.PTransform):
  def __init__(self, database_file, queries):
    self.database_file = database_file
    self.queries = queries

  def expand(self, pcollection):
    return (
        pcollection
        | 'Create None' >> beam.Create(queries)
        | 'SQLite SELECT' >> beam.ParDo(SQLiteSelect(self.database_file))
    )

database_file = 'moon-phases.db'
queries = [
    # (table_name, [column1, column2, ...])
    ('moon_phases', ['phase_emoji', 'peak_datetime', 'phase']),
    ('moon_phases', ['phase_emoji', 'phase']),
]
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read from SQLite' >> SelectFromSQLite(database_file, queries)
      | 'Print rows' >> beam.Map(print)
  )

{'phase_emoji': '🌕', 'peak_datetime': '2017-12-03 15:47:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'peak_datetime': '2017-12-10 07:51:00', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'peak_datetime': '2017-12-18 06:30:00', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'peak_datetime': '2017-12-26 09:20:00', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-02 02:24:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'peak_datetime': '2018-01-08 22:25:00', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'peak_datetime': '2018-01-17 02:17:00', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'peak_datetime': '2018-01-24 22:20:00', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-31 13:27:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌗',

# Writing data

Your might want to write your data in various output formats. Take a look at the
[Built-in I/O Transforms](https://beam.apache.org/documentation/io/built-in)
page for a list of all the available I/O transforms in Beam.

If none of those work for you, you might need to create your own output transform.

> ℹ️ For a more in-depth guide, take a look at the
[Developing a new I/O connector](https://beam.apache.org/documentation/io/developing-io-overview) page.

## Creating an output transform

The most straightforward way to write data would be to use a `Map` transform to write each element into our desired output format. In most cases, however, this would result in a lot of overhead creating, connecting to, and/or deleting resources.

Instead, most data services are optimized to write _batches_ of elements at a time. Batch writes only connects to the service once, and can load many elements at a time.

Here, we discuss two common ways of batching elements for optimized writes: _fixed-sized batches_, and
_[windows](https://beam.apache.org/documentation/programming-guide/#windowing)
of elements_.

## Writing fixed-sized batches

If the order of the elements _is not_ important, we can simply create fixed-sized batches and write those independently.

We can use
[`GroupIntoBatches`](https://beam.apache.org/documentation/transforms/python/aggregation/groupintobatches)
to get fixed-sized batches. Note that it expects `(key, value)` pairs. Since `GroupIntoBatches` is an _aggregation_, all the elements in a batch _must_ fit into memory for each worker.

> ℹ️ `GroupIntoBatches` requires a `(key, value)` pair. For simplicity, this example uses a placeholder `None` key and discards it later. Depending on your data, there might be a key that makes more sense. Using a _balanced_ key, where each key contains around the same number of elements, may help parallelize the batching process.

Let's create something similar to `WriteToText` but keep it simple with a unique identifier in the file name instead of the file count.

In [ ]:
import apache_beam as beam
import glob
import os
import uuid

class WriteBatchesToFiles(beam.PTransform):
  def __init__(self, file_name_prefix, file_name_suffix, batch_size):
    self.file_name_prefix = file_name_prefix
    self.file_name_suffix = file_name_suffix
    self.batch_size = batch_size

  @staticmethod
  def write_file(lines, file_name_prefix, file_name_suffix):
    file_name = f"{file_name_prefix}-{uuid.uuid4().hex}{file_name_suffix}"
    with open(file_name, 'w') as f:
      for line in lines:
        f.write(f"{line}\n")

  def expand(self, pcollection):
    # Remove existing files matching the output file_name pattern.
    for f in glob.glob(f"{self.file_name_prefix}*{self.file_name_suffix}"):
      os.remove(f)
    return (
        pcollection
        # For simplicity we key with `None` and discard it.
        | 'Key with None' >> beam.WithKeys(lambda _: None)
        | 'Group into batches' >> beam.GroupIntoBatches(self.batch_size)
        | 'Discard key' >> beam.Values()
        | 'Write file' >> beam.Map(
            self.write_file,
            file_name_prefix=self.file_name_prefix,
            file_name_suffix=self.file_name_suffix,
        )
    )

output_file_name_prefix = 'outputs/batch'
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Create file lines' >> beam.Create([
          'Each element must be a string.',
          'It writes one element per line.',
          'There are no guarantees on the line order.',
          'The data might be written into multiple files.',
      ])
      | 'Write batches to files' >> WriteBatchesToFiles(
          output_file_name_prefix,
          file_name_suffix='.txt',
          batch_size=3,
      )
  )

In [91]:
# Lets look at the output files and contents.
!head outputs/batch*.txt

==> outputs/batch-92a6a79e31e34fb68db049b78e76b987.txt <==
The data might be written into multiple files.

==> outputs/batch-c6a8494e2a5146949f2a05918b36ee38.txt <==
Each element must be a string.
It writes one element per line.
There are no guarantees on the line order.


## Writing windows of elements

If the order of the elements _is_ important, we could batch the elements by windows. This could be useful in _streaming_ pipelines, where we have an indefinite number of incoming elements and we would like to write windows as they are being processed.

> ℹ️ For more information about windows and triggers, check the [Windowing](https://beam.apache.org/documentation/programming-guide/#windowing) page.

We use a
[custom `DoFn` transform](https://beam.apache.org/documentation/transforms/python/elementwise/pardo/#example-2-pardo-with-timestamp-and-window-information)
to extract the window start time and end time.
We use this for the file names of the output files.

In [84]:
import apache_beam as beam
import datetime
import time

def unix_time(time_str):
  return time.mktime(time.strptime(time_str, '%Y-%m-%d %H:%M:%S'))

class WithWindowInfo(beam.DoFn):
  def process(self, events, window=beam.DoFn.WindowParam):
    yield {
        'events': events,
        'window_start': window.start.to_utc_datetime(),
        'window_end': window.end.to_utc_datetime(),
    }

class WriteWindowsToFiles(beam.PTransform):
  def __init__(self, file_name_prefix, file_name_suffix):
    self.file_name_prefix = file_name_prefix
    self.file_name_suffix = file_name_suffix

  @staticmethod
  def write_file(element, file_name_prefix, file_name_suffix):
    start_date = element['window_start'].date()
    start_time = element['window_start'].time()
    end_time = element['window_end'].time()
    file_name = f"{file_name_prefix}-{start_date}-{start_time}-{end_time}{file_name_suffix}"
    with open(file_name, 'w') as f:
      for event in element['events']:
        event_time = datetime.datetime.fromtimestamp(event['timestamp'])
        f.write(f"{event_time}: {event['event']}\n")

  def expand(self, pcollection):
    return (
        pcollection
        | 'Group all per window' >> beam.GroupBy(lambda _: None)
        | 'Discard key' >> beam.Values()
        | 'Get window info' >> beam.ParDo(WithWindowInfo())
        | 'Write files' >> beam.Map(
            self.write_file,
            self.file_name_prefix,
            self.file_name_suffix,
        )
    )

output_file_name_prefix = 'outputs/window'
window_size_sec = 5 * 60  # 5 minutes
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Create elements' >> beam.Create([
          {'timestamp': unix_time('2020-03-19 08:49:00'), 'event': 'login'},
          {'timestamp': unix_time('2020-03-19 08:49:20'), 'event': 'view_account'},
          {'timestamp': unix_time('2020-03-19 08:50:00'), 'event': 'view_orders'},
          {'timestamp': unix_time('2020-03-19 08:51:00'), 'event': 'track_order'},
          {'timestamp': unix_time('2020-03-19 09:00:00'), 'event': 'logout'},
      ])
      | 'With timestamps' >> beam.Map(
          lambda x: beam.window.TimestampedValue(x, x['timestamp']))
      | 'Fixed-sized windows' >> beam.WindowInto(
            beam.window.FixedWindows(window_size_sec))
      | 'Write windows to files' >> WriteWindowsToFiles(
          output_file_name_prefix,
          file_name_suffix='.txt',
      )
  )

In [92]:
# Lets look at the output files and contents.
!head outputs/window*.txt

==> outputs/window-2020-03-19-08:45:00-08:50:00.txt <==
2020-03-19 08:49:00: login
2020-03-19 08:49:20: view_account

==> outputs/window-2020-03-19-08:50:00-08:55:00.txt <==
2020-03-19 08:50:00: view_orders
2020-03-19 08:51:00: track_order

==> outputs/window-2020-03-19-09:00:00-09:05:00.txt <==
2020-03-19 09:00:00: logout


# What's next?

* [Programming guide](https://beam.apache.org/documentation/programming-guide) -- learn about all the Apache Beam concepts in more depth.
* [Transform catalog](https://beam.apache.org/documentation/transforms/python/overview) -- check out all the available transforms.
* [Mobile gaming example](https://beam.apache.org/get-started/mobile-gaming-example) -- learn more about windowing, triggers, and streaming through a complete example pipeline.
* [Runners](https://beam.apache.org/documentation/runners/capability-matrix) -- check the available runners, their capabilities, and how to run your pipeline in them.